In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
from glob import glob

import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
from tensorflow import keras
from tensorflow.keras import optimizers
import tensorflow as tf
import tensorflow.keras.losses as losses

import albumentations as A

drive_project_root = 'drive/MyDrive/data/'
sys.path.append(drive_project_root)

image_root = 'drive/MyDrive/data/images/'
anno_root = 'drive/MyDrive/data/annotations/'

image_dir = image_root
bbox_dir = anno_root + 'xmls/'    # bounding box
seg_dir = anno_root + 'trimaps/'  # segmentation map

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

!ls

In [ ]:
pip install git+https://github.com/albumentations-team/albumentations.git

In [ ]:
pip uninstall opencv-python

In [ ]:
pip install opencv-python

# 1. 사진 확인
- 1 : 사진에서 개, 고양이에 해당하는 부분 (object)
- 2 : 배경
- 3 : 둘 중 어디에도 속하지 않는 곳 (주로 object의 테두리)

In [ ]:
image_files = glob(image_root+'*.jpg')
image_path = image_files[134]

seg_path = image_path.replace('images', 'annotations/trimaps')
seg_path = seg_path.replace('jpg', 'png')

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
seg_map = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)  # grayscale로 읽기

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.subplot(1, 2, 2)
plt.imshow(seg_map)
plt.show()

목표 : 사진에서 개, 고양이 부분만 예측하는 segmentation model 생성
- seg map에서 1이 아닌 값은 모두 0으로 바꾸기

In [ ]:
seg_map[seg_map != 1] = 0

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.subplot(1, 2, 2)
plt.imshow(seg_map)
plt.show()

---
# 2. Loss 함수의 구현 : IoU / Dice coefficient

- segmentation model 평가할 때 사용
- segmentation은 keras, tensorflow에서 공식적으로 제공하는 loss가 없어 직접 구현 필요

### IoU
- 어차피 사람이 만든 bounding box도 정확하지 않아서 IoU 값이 0.7만 돼도 모델 성능이 좋다고 판단
- 0.9 이상이면 사람보다 잘한다고 판단

### Dice
- F1 Score와 비슷한 개념

문제 : 두 mask가 주어졌을 때 겹치는 부분을 구하려면 각 pixel에 대해서 model이 제대로 예측을 했는지 알아야 함 (true/false)
- 방법 : [특정 threshold를 기준으로 틀리면 0, 맞으면 1] + [step function 사용]
- 한계 : step function은 미분 불가능해서 이 개념을 모델에 바로 사용하기는 어려움
- 해결책
  - 연속적인 함수로 만들기 위해, label의 0/1을 각 pixel이 특정 class에 속할 확률값으로 해석
  - 기존의 binary crossentropy를 사용할 때처럼 모델이 각 pixel을 얼마나 잘 분류했는지 알 수 있게 됨
  - 기존 '맞다/틀리다'에서 '일부분만 맞췄다'는 개념이 추가됨
  - = continuous한 함수가 됨
- 결론 : 학습에 iou, dice를 사용할 수 있게 됨

element wise 곱 + reduce_sum 예시
- y_true = [0, 1, 1, 0]
- y_pred = [0.5, 1, 0.2, 0.3]
- y_true \* y_pred = [0 * 0.5, 1 * 1, 1 * 0.2, 0 * 0.3] = [0, 1, 0.2, 0]
- reduce_sum = 0 + 1 + 0.2 + 0 = 1.2
- 결론 : 두 개는 1.2 pixel이 겹친다

score & loss
- score : 서로 일치할수록 (성능이 좋을수록) 1, 나쁠수록 0에 가까운 값 갖게 됨
- loss : gradient descent를 하기 때문에 성능이 나쁠수록 loss가 커져야 함, 좋을수록 0에 가까운 값

In [ ]:
def iou(y_true, y_pred):
    
    # y_true가 전부 '0'으로 되어 있다면 union이 항상 '0'이 되니 마지막에 union으로 나눌 때 에러 발생
    # 이를 방지하기 위해 smooth 사용
    smooth = 1.

    # Flatten
    # input은 2차원의 이미지인데 더 편리하게 계산하기 위해 보통 1차원 vector로 flatten함
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])

    # 두 tensor를 element-wise 곱셈을 하고 합을 구하면 총 몇 pixel이 겹치는지 알 수 있음
    intersection = tf.reduce_sum(y_true * y_pred)

    # union : 라벨과 예측값의 pixel 개수 모두 더하고 교집합 부분 빼면 됨
    # sum으로 pixel 개수를 구할 수 있는 건 input 값이 0, 1 값으로만 이루어져있기 때문
    # but, y_pred는 0, 1로만 이루어지지 않음 = 'pixel 개수'라고 표현하는 게 맞지는 않음
    # -> 하지만 연속적인 함수로 만들어 주기 위해 어쩔 수 없는 부분
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    
    score = intersection / (union + smooth)
    return score

# dice coefficient : 2 * |X ∩ Y| / (|X| + |Y|)
def dice_coef(y_true, y_pred):

    smooth = 1.

    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])

    intersection = tf.reduce_sum(y_true * y_pred)
    
    score = (2. * intersection) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coef(y_true, y_pred)
    return loss

# binary crossentropy
# - 보통 loss 값 계산할 때 위처럼 dice loss를 단독으로 사용하지 않음
def bce_dice_loss(y_true, y_pred):
    # 보통 1:1 비율로 사용하는데 상황에 따라 10:1, 0.9:1 등으로 적용하기도 함
    loss = 1. * losses.binary_crossentropy(y_true, y_pred) + 1. * dice_loss(y_true, y_pred)
    return loss

### 성능 확인

In [ ]:
# 분모에 smooth라고 안전장치로 1을 더했기 때문에 이 코드의 결과가 1이 나오지는 않음
print('같은 값이 들어온 경우 (dice_coef): ', dice_coef(seg_map.astype('float32'), seg_map.astype('float32')).numpy())
print('같은 값이 들어온 경우 (iou)      : ', iou(seg_map.astype('float32'), seg_map.astype('float32')).numpy())

# prediction으로 모두 0이 들어간 경우 -> 분자인 intersection이 전부 0이니 계산 결과 0 나옴
zeros = np.zeros_like(seg_map)
print('prediction이 0인 경우 (dice_coef): ', dice_coef(seg_map.astype('float32'), zeros.astype('float32')).numpy())
print('prediction이 0인 경우 (iou)      : ', iou(seg_map.astype('float32'), zeros.astype('float32')).numpy())

# prediction으로 모두 1이 들어간 경우
# dice : 0.3949031 -> intersection을 2배 하니 가중치가 더 많이 들어가서 iou보다 큰 값 가짐
# iou : 0.24603069 -> 전체의 24.6% 정도만 겹친다
ones = np.ones_like(seg_map)
print('prediction이 1인 경우 (dice_coef): ', dice_coef(seg_map.astype('float32'), ones.astype('float32')).numpy())
print('prediction이 1인 경우 (iou)      : ', iou(seg_map.astype('float32'), ones.astype('float32')).numpy())

---
# 3. Data Loader의 구현

segmentation task
- 임의의 사진에 대해 개/고양이를 예측하고자 함 (품종 구분X)
- 사진에서 0(배경), 1(개/고양이), 2(테두리) 값 중 1의 부분을 예측

In [ ]:
csv_path = drive_project_root+'kfolds.csv'
df = pd.read_csv(csv_path)
print(df[:10])

idx = random.choice(range(len(df)))  # random하게 하나의 index 선택해서 image 출력하기 위함

file_name = df.loc[idx].file_name
img_path = f'{image_root}{file_name}.jpg'
mask_path = f'{anno_root}trimaps/{file_name}.png'  # segmentation mask

print('img_path: ', img_path)
print('mask_path: ', mask_path)

img = cv2.imread(img_path)  # image 읽어오기
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # mask는 grayscale로 저장되어 있음 -> 옵션 'IMREAD_GRAYSCALE' 입력 필요

# img와 mask는 shape이 같아야 함
# 다만 img는 채널이 3개, mask는 채널이 1개
assert img.shape[:2] == mask.shape[:2]  # 가로, 세로 길이 일치여부 확인

# image 시각화
plt.figure(figsize=(10, 10))
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(mask)
plt.show()

### 3.1 Augmentation

In [ ]:
class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            # Declare an augmentation pipeline
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5),  # 좌우반전
                A.ShiftScaleRotate(       # 상하좌우 이동 (shift) + 확대/축소 (scale) + 회전 (rotate)
                    p=0.5,
                    shift_limit=0.05,  # 이미지의 가로 길이가 최대 몇% 넘어가도 되는지
                    scale_limit=0.05,  # 최대 몇% 확대/축소할지
                    rotate_limit=15,   # 최대 몇 도 회전할지
                ),
                A.CoarseDropout(       # 이미지에 구멍을 뚫기
                    p=0.5,
                    max_holes=8,                 # 최대 구멍 개수
                    max_height=int(0.1 * size),  # 가로 최대 길이 : 이미지의 10%
                    max_width=int(0.1 * size),
                ),
                A.RandomBrightnessContrast(p=0.2),
            ])

    def __call__(self, **kwargs):
        if self.transform:    # train mode인 경우
            augmented = self.transform(**kwargs)
            img = augmented['image']
            mask = augmented['mask']   # mask도 image와 동일한 rotate, scale 등을 적용하기 위해 추가
            return img, mask

### 3.2 Data Generator

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, image_size,
                 fold, mode='train', shuffle=True):
        
        self.batch_size = batch_size
        self.image_size = image_size
        self.fold = fold
        self.mode = mode
        self.shuffle = shuffle

        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        #### Remove invalid files
        #### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]

        self.df = self.df[~self.df['file_name'].isin(invalid_filenames)]
        self.transform = Augmentation(image_size, mode)

        self.on_epoch_end()
            
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
        
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        return np.array(batch_x), np.array(batch_y)
        
    # 이때 label == mask
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for _, r in data.iterrows():
            file_name = r['file_name']
            
            image = cv2.imread(f'{image_root}{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.image_size, self.image_size))

            mask = cv2.imread(f'{anno_root}trimaps/{file_name}.png', cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (self.image_size, self.image_size))  # mask도 imge에 맞게 resize 필요

            # 0, 1, 2 중 1(개/고양이)에 해당하는 값들만 예측하는 task
            # -> 1이 아닌 값들은 모두 날려줘야 함
            mask[mask != 1] = 0

            # ★ 예측 시 주의 : image에 적용되는 operation들이 mask에도 적용되어야 함
            # i.e. image가 15도 회전되었다면 mask도 이에 맞춰 15도 회전 필요
            # -> 방법 : image와 mask를 함께 transform에 넘겨주기
            if self.mode == 'train': 
                # augmentation 중에는 image가 uint8인 경우에만 적용되는 것들이 있음
                # -> image를 먼저 uint8로 변환하기
                image = image.astype('uint8')
                image, mask = self.transform(image=image, mask=mask)

            image = image.astype('float32')
            image = image / 255.
            mask = mask.astype('float32')
            
            # label = int(r['species']) - 1
            
            batch_x.append(image)
            batch_y.append(mask)
        
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

train_generator = DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

valid_generator = DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=128,
    image_size=128,
    shuffle=True
)

시각화를 통해 모델 생성이 잘 되었는지 확인

In [ ]:
for batch in train_generator:
    X, y = batch
    plt.figure(figsize=(10, 10))

    for i in range(3):
        plt.subplot(3, 2, i*2 + 1)
        plt.imshow(X[i])   # image
        plt.axis('off')
        plt.subplot(3, 2, i*2 + 2)
        plt.imshow(y[i], cmap='gray')   # mask
        plt.axis('off')
        plt.tight_layout()
    break

---
# 4. UNet

- 목적
  - biomedical 분야에서 image segmentation을 목적으로 제안된 end to end 방식의 fully convolutional network 모델
  - 의료 영상에 적용하기 위해 만들어졌지만 일반적인 이미지들에 대해서도 좋은 성능을 보이기 때문에 지금까지도 널리 쓰이는 model architecture
- U처럼 생긴 네트워크의 구성 때문에 'UNet'이라고 불리게 됨
- 구조
  - 1) Contraction path (encoder) : image의 context 학습
    - convolution layer, maxpooling layer의 stack으로 이루어짐
  - 2) Expanding path (decoder) : image localization 학습 (위치 정보)
    - transpose convolution 사용
  - 즉, 처음부터 끝까지 convolution layer로만 이루어짐
  - = Dense layer 없음
    - input image size에 영향 받지 않게 됨
    - 물론 그렇다고 모든 image size를 input으로 받을 수는 없음
    - encoder에서 down sampling이 maxpooling 때문에 한 번씩 발생할 때마다 가로와 세로 길이가 딱 절반씩 감소하는데 현재 구조에서는 총 4번 down sampling이 발생함 -> input image size가 최소 16 * 16은 되어야 함 (16 \* 16보다 작으면 error 발생)
- output : image와 같은 크기의 segmentation mask
- encoder가 image context를 학습하는 방법
  - Max Pooling (Down Sampling)
    - feature map의 크기를 줄여 최종적으로 network의 parameter 수를 감소시키는 것
    - = 중요한 정보만 유지하고 나머지는 버림
    - 중요한 정보 : image의 context를 잘 설명하는 값 (i.e. 최대 pixel)
- decoder (Up Sampling)
  - image segmentation을 수행하면 image의 context 뿐만 아니라 localization도 함께 수행되어야 하기 때문에 잃어버린 위치에 대한 정보를 학습하는 역할
  - Transposed Convolution (Deep Convolution)
    - Up Sampling을 위해 가장 자주 사용되는 방법
    - 학습 가능한 parameter들로 up sampling을 수행하는 layer


### 4.1 Training with UNet

- keras에서 segmentation model을 따로 제공하지 않기 때문에 쓰고 싶은 모델이 있으면 그 모델이 구현되어 있는 유명한 repo를 가져와서 수정해서 사용하는 방식이 일반적
- [UNet 구현 Repo](https://github.com/karolzak/keras-unet)
  - UNet은 예전에 나온 논문이기 때문에 요즘 default로 많이 쓰이는 dropout, batch normalization 등이 적용 안 되어 있음
  - 그런데 이 repository에서는 이런 부분까지 적용할 수 있게 해 놓음 (Customizable U-Net) -> 이 부분 가져다 쓰기

In [ ]:
pip install git+https://github.com/karolzak/keras-unet

In [ ]:
from keras_unet.models import custom_unet

model = custom_unet(
    input_shape=(128, 128, 3),
    # input_shape=(32, 32, 3),  # OOM error 때문에 input shape 줄여봄
    use_batch_norm=True,
    upsample_mode='deconv', # simple : 학습이 일어나지 않는 것 -> deconv를 쓰는 게 일반적
    dropout_type='spatial', # spatial을 권장
    # use_dropout_on_upsampling=False, # upsampling할 때 dropout 사용할 지
    use_attention=True,  # encoder와 decoder 사이 skip-connection을 지나갈 때 attention을 사용할 지
    num_classes=1,
    filters=64,          # (default: 16)
    dropout=0.2,
    num_layers=4, # encoder에 있는 layer 개수 (convolution에 몇 개 block을 만들지) - 깊다고 항상 좋은 건 아님
    output_activation='sigmoid' # 이진분류
)

model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[iou])
model.summary()

- custom_unet이 인자로 받는 것들의 각각의 의미

In [ ]:
help(custom_unet)

- 10번째 epoch에서 val_iou가 0.73인데 이 정도면 어느정도 모양은 다 알아볼 수 있다고 생각할 수 있음
- 0.85 이상이면 정말 잘한다
- 0.90 이상이면 사람보다 잘한다

에러발생 : OOM (out of memory)
- You can fix this by making your model smaller or reducing your batch size
  - 이미지 input_shape을 줄이거나 batch_size 줄일 것
- If you have multiple GPUS at hand, kindly select a GPU which is not as busy as this one
  - `export CUDA_VISIBLE_DEVICES=1`
  - check the available GPUs using `nvidia-smi `

In [ ]:
!export CUDA_VISIBLE_DEVICES=1

In [ ]:
!nvidia-smi 

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

### 4.2 시각화

- IoU : 우상향 -> 단순히 epoch 수만 더 늘려도 성능이 올라갈 수 있음

In [ ]:
history = history.history

plt.figure(figsize=(10, 5))
plt.subplot(1,2,1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss')

plt.subplot(1,2,2)
plt.plot(history['iou'], label='train')
plt.plot(history['val_iou'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('iou')
plt.title('IoU')
plt.show()

In [ ]:
for images, masks in valid_generator:
    break

In [ ]:
preds = model.predict(images)

In [ ]:
import random
idx = random.choice(range(len(preds)))
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(images[idx,...])
plt.subplot(1,2,2)
plt.imshow(images[idx,...,0], cmap='gray')

---
# 5. TensorBoard visualization

- Google에서 open-source로 제공하는 학습/모델 시각화 tool
- 장점
  - 학습 도중에도 loss/accuracy를 그래프로 확인 가능
  - 기존처럼 학습이 완료된 후 epoch에 따른 loss, iou 변화를 따로 그래프로 시각화하지 않아도 됨
  - 학습이 진행됨에 따라 특정 이미지에 대해 모델의 예측 결과가 어떻게 달라지는 지도 확인 가능 -> 매우 유용
- keras에서는 tensorboard logging을 callback으로 제공 -> 기존과 학습 code는 동일하게 하고 callback 함수만 추가 필요

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import Callback

### 5.1 custom callback 만들기

- https://www.tensorflow.org/tensorboard/image_summaries
  - plot_to_image, image_grid 함수 코드 그대로 복사 + 붙여넣기

In [ ]:
# 임의의 figure에 대해 변환하는 함수라 추가 수정 필요 없음
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

# 전체 수정 필요
def image_grid(img, mask, preds):

    figure = plt.figure(figsize=(10, 10))

    # 처음 3개의 이미지에 대해서만 시각화하고자 함
    for i in range(3):
        plt.subplot(3, 3, 3*i + 1)
        plt.imshow(img[i])
        plt.axis('off')
        plt.title('img')

        plt.subplot(3, 3, 3*i + 2)  # 오른쪽에 위치 (정답 mask)
        plt.imshow(mask[i], camp='gray')
        plt.axis('off')
        plt.title('gt')

        plt.subplot(3, 3, 3*i + 3)  # 예측 결과
        plt.imshow(preds[i, ..., 0], camp='gray')  # i번째 이미지, 마지막 채널은 single 채널이라 의미가 없으니 없애주기 -> gray
        plt.axis('off')
        plt.title('pred')

    return figure

In [ ]:
# 매 epoch이 끝날 때마다 사용자가 지정해 준 이미지에 대한 모델의 예측 결과를 tensorboard에 logging 하는 callback 함수
class TrainHistory(Callback):
    def __init__(self, data, log_dir=drive_project_root+'weights'):
        self.img, self.mask = data    # 입력 데이터 -> data generator에서 가져오기
        self.write = tf.summary.create_file_writer(log_dir)
        
    # image logging 주기 정하기
    # 매 batch 마다 logging 하면 학습 느려짐 -> 매 epoch이 "끝날 때마다" logging하기
    def on_epoch_end(self, epoch, logs=None):
        pred = self.model.predict(self.img)   # 예측 결과
        
        figure = image_grid(self.img, self.mask, pred)  # matplotlib로 plot 해주는 함수

        with self.writer.as_default():
            tf.summary.image(
                'plot',
                plot_to_image(figure),  # tensorboard에 logging이 가능한 image 형태로 변환
                epoch                   # 몇 epoch에 대한 결과인지
            )

In [ ]:
callbacks = [
    TensorBoard(drive_project_root+'weights/'),  # 저장위치
    TrainHistory(
        log_dir=drive_project_root+'weights/',
        data=valid_generator[0][:3]  # 첫 번째 batch에 있는 처음 3개의 이미지 = 고정된 이미지에 대한 결과가 어떻게 변화하는지 확인하기 위함
    )
]

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1,
    callbacks=callbacks
)

### 5.2 Tensorboard 띄우기

- 상단 'IMAGES' 들어가서 step 막대기 조절하며, epoch이 지날 때마다 (학습이 진행됨에 따라) 예측 변화 정도 확인해 보기

In [ ]:
%load_ext tensorboard
%tensorboard --logidr drive/MyDrive/data/weights/

---
# 6. Learning Rate Schedule

- optimizer에서 learning rate를 설정하지 않으면 default인 0.001이 적용됨
- schedule : 학습이 진행됨에 따라 learning rate를 변환하는 것

In [ ]:
# 한 epoch을 학습하는 동안 몇 번의 weight update를 하는지
# 전체 데이터 개수 / batch_size
len(train_generator)

### 6.1 Exponential Decay : 10 epoch에 대한 시각화
- decay_steps = 46 : exponential 이라는 이름과는 맞지 않게 linear 하게 감소함
- decay_steps = 3 : 0.96이 더 자주 곱해지니 learning rate가 더 빠르게 감소함
  - 여기서는 보여주기 위해 3으로 설정했지만, 일반적으로 3으로 설정하는 건 너무 작은 편

In [ ]:
def decayed_learning_rate(step):
    initial_learning_rate = 0.001

    # 위에서 len(train_generator) = 46이었는데 decay_steps도 46으로 설정하면
    # 한 epoch이 지날 때마다 learning rate도 4% 씩 줄이라는 의미 (0.96)
    decay_rate = 0.96
    # decay_steps = 46 
    decay_steps = 3

    return initial_learning_rate * decay_rate ** (step / decay_steps)

# 한 번 학습을 진행할 때마다 46 step이 있음
# 10 epoch동안 학습하면 46 * 10
lrs = [decayed_learning_rate(i) for i in range(1, 46 * 10)]
plt.plot(lrs)

### 6.2 Optimizer에 적용하기

In [ ]:
lr_schedule = optimizers.schedules.ExponentialDecay(
    0.001,  # initial learning rate
    decay_steps=3,
    decay_rate=0.96,  # 3 step마다 0.96 곱하기
)
optimizer = optimizers.Adam(lr_schedule)

### 6.3 Modeling

In [ ]:
model = custom_unet(
    input_shape=(128, 128, 3),
    use_batch_norm=True,
    num_classes=1,
    filters=64,
    dropout=0.2,
    use_attention=True,
    output_activation='sigmoid'
)

model.compile(optimizer=optimizer, loss=bce_dice_loss, metrics=[iou])
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

### 6.4 visualization : train data

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses[:10], label='w/o scheduling')
plt.plot(history.history['loss'], label='scheduling')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Train Loss")

plt.subplot(1, 2, 2)
plt.plot(train_ious[:10], label='w/o scheduling')
plt.plot(history.history['iou'], label='scheduling')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('IoU')
plt.title("Train IoU")
plt.show()

### 6.5 visualization : validation data

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(valid_losses[:10], label='w/o scheduling')
plt.plot(history.history['val_loss'], label='scheduling')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Valid Loss")

plt.subplot(1, 2, 2)
plt.plot(valid_ious[:10], label='w/o scheduling')
plt.plot(history.history['val_iou'], label='scheduling')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('IoU')
plt.title("Valid IoU")
plt.show()

---
# 7. Multiclass segmentation

- 3가지 부분(배경, 개/고양이, 테두리)을 모두 예측하는 모델 만들기
- Augmentation은 binary와 같이 써도 되고, DataGenerator 부분만 일부 수정 필요

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, image_size,
                 fold, mode='train', shuffle=True):
        
        self.batch_size = batch_size
        self.image_size = image_size
        self.fold = fold
        self.mode = mode
        self.shuffle = shuffle

        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        #### Remove invalid files
        #### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]

        self.df = self.df[~self.df['file_name'].isin(invalid_filenames)]
        self.transform = Augmentation(image_size, mode)

        self.on_epoch_end()
            
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
        
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        return np.array(batch_x), np.array(batch_y)
        
    # 이때 label == mask
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for _, r in data.iterrows():
            file_name = r['file_name']
            
            image = cv2.imread(f'{image_root}{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.image_size, self.image_size))

            mask = cv2.imread(f'{anno_root}trimaps/{file_name}.png', cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (self.image_size, self.image_size))

            # mask[mask != 1] = 0    -> 개/고양이 부분이 아닌 다른 부분을 모두 0으로 바꿀 필요 없음
            # 단, 현재 class는 1, 2, 3인데 keras에서는 0, 1, 2 순이 되어야 하기 때문에 변경 필요
            mask -= 1                 # 개/고양이(0), 배경(1), 테두리(2)

            if self.mode == 'train': 
                image = image.astype('uint8')
                image, mask = self.transform(image=image, mask=mask)

            image = image.astype('float32')
            image = image / 255.
            mask = mask.astype('float32')
            
            # label = int(r['species']) - 1
            
            batch_x.append(image)
            batch_y.append(mask)
        
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

train_generator = DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

valid_generator = DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=128,
    image_size=128,
    shuffle=True
)

In [ ]:
model = custom_unet(
    input_shape=(128, 128, 3),
    use_batch_norm=True,
    num_classes=3,   ### 수정
    filters=64,
    dropout=0.2,
    use_attention=True,
    output_activation='softmax' ### 수정
)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')  # loss 부분 수정 필요
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,
    verbose=1
)

In [ ]:
history = history.history
plt.figure(figsize=(10, 5))
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss')
plt.show()

- 모델이 잘 예측하고 있는지 시각화

In [ ]:
for images, masks in valid_generator:
    break

preds = model.predict(images)
pred.shape   # 채널 : 3

- 3개의 class 중 확률이 가장 높은 1개 값으로 분류해서 출력하기 (나머지는 버리기)
- 현재는 각 class(0, 1, 2)에 속할 확률이 모두 표시되어 있음

In [ ]:
pred = np.argmax(preds[idx], axis=-1) 
np.unique(pred)   # 0, 1, 2 값만 나옴

In [ ]:
idx = random.choice(range(len(preds)))
plt.figure(figsize=(10, 10))
plt.subplot(1, 3, 1)
plt.imshow(images[idx, ...])  # 원본
plt.subplot(1, 3, 2)
plt.imshow(masks[idx])   # mask
plt.subplot(1, 3, 3)
plt.imshow(preds)        # 예측값